In [1]:
import pandas as pd
import numpy as np

In [2]:
merged_df = pd.read_csv('data/filtered/merged_valais.csv')

In [ ]:
#Highest precipitation over 10 minutes
merged_df.sort_values('precip', ascending=False).head(10)

,station,time,precip,moisture,pression,temperature,North,East,name,east,north,altitude,Canton
440815,VSARO,202205200510,44.6,NaN,NaN,NaN,NaN,NaN,Arolla,603508,95832,2005,VS
468275,VSARO,202211262150,36.4,NaN,NaN,NaN,NaN,NaN,Arolla,603508,95832,2005,VS
468230,VSARO,202211261420,36.4,NaN,NaN,NaN,NaN,NaN,Arolla,603508,95832,2005,VS
3237100,VSEMO,202007212040,22.6,NaN,NaN,NaN,NaN,NaN,Emosson,560664,101691,1923,VS
32069,VSANZ,201908111650,21.5,NaN,NaN,NaN,NaN,NaN,Anzère,597607,128205,1614,VS
2549326,VSCLU,202206231540,21.4,NaN,NaN,NaN,NaN,NaN,Clusanfe,557237,109533,1926,VS
2503988,VSCLU,202108121920,18.8,NaN,NaN,NaN,NaN,NaN,Clusanfe,557237,109533,1926,VS
4239179,FIT,201908111750,17.9,NaN,NaN,NaN,NaN,NaN,Fieschertal,653990,142082,1175,VS
3237236,VSEMO,202007221920,17.7,NaN,NaN,NaN,NaN,NaN,Emosson,560664,101691,1923,VS
2398561,VSCLU,201908111610,17.2,NaN,NaN,NaN,NaN,NaN,Clusanfe,557237,109533,1926,VS


In [5]:
precip_copy = merged_df.copy()

precip_copy['dry'] = (precip_copy['precip']==0).astype(int)

precip_copy['dry_group'] = precip_copy.groupby('station')['dry'].diff().ne(0).cumsum()

dry_periods = precip_copy[precip_copy['dry'] == 1].groupby(['station', 'dry_group'])['time'].agg(['min', 'max', 'count']).reset_index()

output = dry_periods.sort_values('count', ascending=False)

print(output.head(5).to_latex(index=False, caption="Top 5 Longest Dry Periods", label="tab:dry_periods"))

\begin{table}
\caption{Top 5 Longest Dry Periods}
\label{tab:dry_periods}
\begin{tabular}{lrrrr}
\toprule
station & dry_group & min & max & count \\
\midrule
ULR & 424354 & 202003130150 & 202004191740 & 5424 \\
VSSAB & 339475 & 202202220840 & 202203310610 & 5314 \\
MAB & 267188 & 202003201820 & 202004261440 & 5307 \\
ZER & 458502 & 202202211940 & 202203300620 & 5249 \\
SIO & 388702 & 202301180130 & 202302230110 & 5183 \\
\bottomrule
\end{tabular}
\end{table}



In [9]:
#highest amount of precipitation
precip_sum = precip[['station', 'precipitation']].groupby('station').sum()
print("Station with the highest total precipitation:")
print(precip_sum.sort_values('precipitation', ascending=False).head(1))

print("Station with the lowest total precipitation:")
print(precip_sum.sort_values('precipitation', ascending=False).tail(1))

#precipitation mean()
precip_sum = precip[['station', 'precipitation']].groupby('station').mean()
print("Station with the highest average precipitation:")
print(precip_sum.sort_values('precipitation', ascending=False).head(1))

print("Station with the lowest average precipitation:")
print(precip_sum.sort_values('precipitation', ascending=False).tail(1))

Station with the highest total precipitation:
         precipitation
station               
SAE            16475.4
Station with the lowest total precipitation:
         precipitation
station               
SNG             1299.6
Station with the highest average precipitation:
         precipitation
station               
SAE           0.062657
Station with the lowest average precipitation:
         precipitation
station               
VSMAT         0.010165


In [5]:
#Highest temperature recorded
temperature.sort_values('temperature', ascending=False).head(10)

,station,time,temperature
11550896,GVE,202308241320,39.3
11550901,GVE,202308241410,39.1
11550897,GVE,202308241330,38.7
11550902,GVE,202308241420,38.7
11550899,GVE,202308241350,38.7
11550900,GVE,202308241400,38.5
22594549,CGI,202308241410,38.5
11550898,GVE,202308241340,38.4
11550891,GVE,202308241230,38.4
22594548,CGI,202308241400,38.4


In [6]:
#Lowest temp
temperature.sort_values('temperature').head(10)

,station,time,temperature
26461989,SAM,202102140330,-30.5
4318737,BUF,202102140530,-29.7
26457094,SAM,202101110340,-29.6
26462010,SAM,202102140700,-29.5
4318745,BUF,202102140650,-29.5
4318744,BUF,202102140640,-29.5
4318717,BUF,202102140210,-29.4
26462000,SAM,202102140520,-29.4
4318734,BUF,202102140500,-29.3
26461998,SAM,202102140500,-29.2


In [ ]:
temperature.index = pd.to_datetime(temperature['time'], format='%Y%m%d%H%M')
precip.index = pd.to_datetime(precip['time'], format='%Y%m%d%H%M')
wind.index = pd.to_datetime(wind['time'], format='%Y%m%d%H%M')
moisture.index = pd.to_datetime(moisture['time'], format='%Y%m%d%H%M')
pression.index = pd.to_datetime(pression['time'], format='%Y%m%d%H%M')

merged_data = precip[['precipitation']].merge(temperature[['temperature']], left_index=True, right_index=True, how='left')
merged_data = merged_data.merge(wind[['North', 'East']],  left_index=True, right_index=True, how='left')
merged_data = merged_data.merge(moisture[['moisture']],  left_index=True, right_index=True, how='left')
merged_data = merged_data.merge(pression[['pression']],  left_index=True, right_index=True, how='left')

: 